# taller PyTOUGH - Ejercicio 7

Ejercicio tomado de <a href="https://github.com/fguemar/tallerPyTOUGH">Taller PyTOUGH</a> por Fernando J. Guerrero, con el uso de biblioteca de funcioes <a href="https://github.com/acroucher/PyTOUGH">PyTOUGH</a> de Adrian Croucher. Uso bajo licencia <a href="https://creativecommons.org/licenses/by-nc-nd/4.0?ref=chooser-v1">Attribution-NonCommercial-NoDerivatives 4.0 International</a>

In [1]:
#==== PyTOUGH ==== 
#    geometry
from mulgrids import *
#   tough2 grids
from t2grids import *
# tough2 data files
from t2data import *
#  postprocesing
from t2listing import *
#================= 

# calls to system
import os
# Visualization Toolkit
from vtk import *

import datetime
#from sympy import *
import math

In [2]:
#=================================== *** ====================================
#guarda la fecha de hoy en la variable 'now'
now = datetime.datetime.now()
#guarda en 'cwd' la dirección del directorio de trabajo
cwd=os.getcwd()

In [3]:
gmshgeo=mulgrid().from_gmsh('miMallaGMSH.msh', [1.], convention=0, atmos_type=2, top_elevation=0., chars = ascii_lowercase, spaces=True, block_order=None)
gmshgeo

267 nodes; 476 columns; 2 layers; 476 blocks; 0 wells
Naming convention: 0 (3 characters for column, 2 digits for layer)
Atmosphere type  : 2 (no atmosphere blocks)

In [4]:
#carga las elevaciones de la malla voronoi
surFinput=np.loadtxt('elevations_Voronoi.txt')
min(surFinput[:,2])

216.0

In [56]:
# Write 'in' file for AMESH and run amesh
f=open(cwd+'/amesh/in','w')
f.write('locat\n')

tol=0.1
thck=180.
z0=min(surFinput[:,2])
zc=z0+thck*0.5
i=1
for ly in range(1,14):
    zc=zc-thck
    for node in gmshgeo.nodelist[:]:
        xi=node.pos[0]
        yi=node.pos[1]
        f.write(format(i,"5d")+format(ly,"5d")+format(xi,"20.10e")+format(yi,"20.10e")+format(zc,"20.10e")+format(thck,"20.10e")+'\n')
        i=i+1
f.write('\n')

f.write('toler\n')
f.write(format(tol,"10.4f")+'\n')
f.write('\n')

#corners of the polygon - clockwise
f.write('bound\n')
f.write(format(0.,'9.3f')+' '+format(1000.,'9.3f')+'\n')
f.write(format(0.,'9.3f')+' '+format(15000.,'9.3f')+'\n')
f.write(format(2000.,'9.3f')+' '+format(16000.,'9.3f')+'\n')
f.write(format(10000.,'9.3f')+' '+format(16000.,'9.3f')+'\n')
f.write(format(12000.,'9.3f')+' '+format(15000.,'9.3f')+'\n')
f.write(format(12000.,'9.3f')+' '+format(1000.,'9.3f')+'\n')
f.write(format(10000.,'9.3f')+' '+format(0.,'9.3f')+'\n')
f.write(format(2000.,'9.3f')+' '+format(0.,'9.3f')+'\n')

f.close()

#>>>>>>>>>>>>>>>>>>>ATENCIÓN>>>>>>>>>>>>>>>>>>>>>>>>>>>
#  con esta instrucción se ejecuta AMESH en mac y se mueven los archivos in y segmnt un directorio atrás
#  revisa cómo puedes ejecutar el software en tu sistema
#>>>>>>>>>>>>>>>>>>>ATENCIÓN>>>>>>>>>>>>>>>>>>>>>>>>>>>
os.system('cd amesh;./amesh6_osx;mv in segmt ..')

0

In [5]:
#build geometry from amesh mesh
#use convention=2 for definition of GENER SL,NS - NS given by column name
#0 - 3 characters for column followed by 2 digits for layer
#1 - 3 characters for layer followed by 2 digits for column
#2 - 2 characters for layer followed by 3 digits for column
amshgeo=mulgrid().from_amesh(input_filename='in', segment_filename='segmt', convention=0, node_tolerance=None, justify='r', chars=ascii_lowercase, spaces=True, block_order =None)
geo=amshgeo[0]

#un nodo atmosférico sobre cada columna
geo.atmosphere_type=1

#asignamos elevaciones a las columnas
i=0 
for cl in geo.columnlist[:]:
    cl.surface=surFinput[i,2]
    i=i+1

In [6]:
geo.write_vtk(filename='miMalla3D.vtu', arrays=None, wells=False, blockmap={}, surface_snap =0.1)

In [7]:
#guarda las columnas que pertenecen a la frontera
boundary_cls = [cl for cl in geo.columnlist if cl.centre[1]==0. in geo.boundary_columns]


southcols=

In [12]:
#=============== ROCK TYPES ===============
#Definición de tipos de roca
grani=rocktype(name = 'grani', density=2700., porosity=0.005, permeability = [1.e-19, 1.e-19, 1.e-19], conductivity=2.45, specific_heat = 950.) 
prcom=rocktype(name = 'prcom', density=2600., porosity=0.08, permeability = [1.e-16, 1.e-16, 1.e-16], conductivity=2.51, specific_heat = 920.) 
cmndu=rocktype(name = 'cmndu', density=2057., porosity=0.03, permeability = [1.e-17, 1.e-17, 1.e-17], conductivity=2.51, specific_heat = 920.) 
bsalt=rocktype(name = 'bsalt', density=2057., porosity=0.01, permeability = [4.e-19, 4.e-19, 4.e-19], conductivity=2.51, specific_heat = 920.) 
domes=rocktype(name = 'domes', density=2057., porosity=0.03, permeability = [6.e-19, 6.e-19, 6.e-19], conductivity=2.51, specific_heat = 920.) 
#               *** ATMOS ***
atmos=rocktype(name = 'atmos', density=0.94,porosity=0.99, permeability = [1.e-12, 1.e-12, 1.e-12], conductivity=2.51, specific_heat =1.E5) 
atmos.relative_permeability['type']=1
atmos.relative_permeability['parameters']=[0.01,0.,1.,.99]
atmos.capillarity['type']=1
atmos.capillarity['parameters']=[0.,0.,1.] #truns out cp=0.
atmos.tortuosity=1.
atmos.nad=2

#crea una lista con los tipos de roca
ltv_rocktypes=[grani,prcom,cmndu,bsalt,domes,atmos]

In [13]:
#Crea un objeto t2geo a partir de la geometría creada con AMESH
t2geo=t2grid().fromgeo(geo) 
#t2geo.check(fix=False,silent=False)

In [14]:
#agrega los tipos de roca a t2geo
for rck in ltv_rocktypes:
    t2geo.add_rocktype(rck)
                    
# Distribuye los tipos de roca ene el dominio
for blk in t2geo.blocklist[:]:
    if blk.atmosphere == True:
        blk.rocktype = atmos
        blk.volume = 1.E50
    elif blk.centre[2] <= -900.:
        blk.rocktype = grani
    elif blk.centre[2] > -900. and blk.centre[2] <= -700.:
        blk.rocktype = prcom
    elif blk.centre[2] > -700. and blk.centre[2] <= -500.:
        blk.rocktype = cmndu
    elif blk.centre[2] > -500. and blk.centre[2] <= -300.:
        blk.rocktype = bsalt
    else:
        if blk.atmosphere == False:
            blk.rocktype = domes


In [16]:
t2geo.write_vtk(geo,'miMalla3DTOUGH2.vtu', wells=False, blockmap = {}, surface_snap=0.1)

In [33]:
#Crea un objeto t2data y asígnale bloques de datos de flow.inp
#------------- EOS3 -------------
dat=t2data(); dat.grid=t2geo #set the mesh from t2geo object
dat.title=now.strftime('# %Y-%m-%d:')+' EOS3 LTV model by FGuerrero et al.'
dat.multi['num_components']=2
#en este modelo se parte de 'num_equations'=3 - no isotérmico. Lo usual es iniciar con un modelo sin
#transporte de calor y solo buscar las presiones hidrostáticas, una estrategia más cautelosa. 
dat.multi['num_equations']=3 #eos1: isothermal 2, non isothermal 3
dat.multi['num_phases']=2
dat.multi['num_secondary_parameters']=6
#--------------TIMES-------------
dat.output_times['num_times_specified']=8
yrs=3600.*24.*365.; dat.output_times['time']=[yrs*1,yrs*2,yrs*3,yrs*4,yrs*5,yrs*6,yrs*7,yrs*8]

#--------------PARAM-------------
dat.parameter['max_iterations']=8    #default is 8
dat.parameter['newton_weight']=1.    #default is 1
#      ...      Output     ...
dat.parameter['print_level']=2 #2: KDATA print mass and heat fluxes and flow velocities
dat.parameter['print_interval']=10 #printout will occur for every multiple of MCYPR steps
dat.parameter['max_timesteps']=9999 # MCYC maximum number of time steps to be calculated.
#      ---      MOP     ---
dat.parameter['option'][1]=1    #1: a short printout for non-convergent iterations
dat.parameter['option'][5]=3    #which EOS
#dat.parameter['option'][15]=    #Semi-analytical heat exchange
dat.parameter['option'][16]=4    #provides automatic time step control
dat.parameter['option'][21]=2    #2: bi-conjugate gradient solver. 3: DSLUCS (default).
dat.parameter['relative_error']=1.E-5 # convergence criterion for relative error (default: 1.E-5)
#      ...    SIM TIME     ...
dat.parameter['tstart']=0. #starting time of simulation in seconds
#............................................................
#dat.parameter['tstop']=(3600.*24.*365.)*100. #<<<<<<TIMAX<<<<<<
#............................................................
dat.parameter['const_timestep']=1. #DELTEN: length of time steps in seconds
#dat.parameter['max_timestep']=(3600.*24.*365.)*10.  
dat.parameter['gravity']=9.81  

#============= INITIAL CONDITIONS =================
strt=0
if strt == 1:
#                >>> con START sin INCON <<<
    #Esquema de inicialización por default - ASEGÚRATE DE NO TENER ARCHIVO 'INCON'
    #dat.start=True; dat.parameter['default_incons']=[1.013E5, 0., 2.]

    #Los bloques atmosféricos se inician según su elevación - permanecen constantes
    #atmosly=geo.layerlist[0]
    #for cl in geo.columnlist[:]:
    #    blk=t2geo.block[geo.block_name(atmosly.name, cl.name)]
    #    p=1.013E5-(11.295*cl.surface) #Barry and Chorley (2004)
    #    tm=30.-(6.5E-3*cl.surface)
    #    dat.incon[blk.name]=[None,[p, 10.99, tm]]

    #===========  INITIAL GENER  ============
    dat.clear_generators()
    bly=geo.layerlist[-1] # bottom layer
    cols=[cl for cl in geo.columnlist]

    q=90.E-3 # 90 mW/m2
    for cl in cols:
        blockname = geo.block_name(bly.name, cl.name)
        #generator name: three arbitrary charactes + two digits. Consistency with NAMING CONVENTION!
        gen = t2generator(name =cl.name+bly.name, block = blockname, type ='HEAT', gx = q*cl.area)
        dat.add_generator(gen)
    #========================================
else:
#                >>> con INCON - sin START <<<
    os.system('mv SAVE INCON')
    dat.start=False
        # DEFINE LATERAL BLOCKS AS BOUNDARY BLOCKS
    for cl in boundary_cls:
        for ly in geo.layerlist[1:]:
            blk=t2geo.block[geo.block_name(ly.name, cl.name)]
            blk.volume=1.e50
            
    for con in t2geo.connectionlist[0:]:
        if con.block[0].volume == 1.e50 and con.block[1].volume == 1.e50:
            t2geo.delete_connection((con.block[0].name,con.block[1].name))
        elif con.block[0].volume == 1.e50 and con.block[1].volume != 1.e50:
            con.distance[0]=1.e-6
        elif con.block[0].volume != 1.e50 and con.block[1].volume == 1.e50:
            con.distance[1]=1.e-6
    #===========  CONTIN GENER  ============
    os.system('rm GENER')
    dat.clear_generators()
    bly=geo.layerlist[-1] # bottom layer
    cols=[cl for cl in geo.columnlist]
    #Heat flow
    q=90.E-3 # 90 mW/m2
    for cl in cols:
        if cl in boundary_cls:
            continue
        blockname = geo.block_name(bly.name, cl.name)
        #generator name: three arbitrary charactes + two digits. Consistency with NAMING CONVENTION!
        gen = t2generator(name =cl.name+bly.name, block = blockname, type ='HEAT', gx = q*cl.area)
        dat.add_generator(gen)
     #Mass flow
    m=2.50E-3 # kg/s
    cols=[cl for cl in geo.columnlist if abs(cl.centre[0]-geo.centre[0]) < 2000. and \
    abs(cl.centre[1]-geo.centre[1]) < 2000.]
    for cl in cols:
        if cl in boundary_cls:
            continue
        blockname = geo.block_name(bly.name, cl.name)
        #generator name: three arbitrary charactes + two digits. Consistency with NAMING CONVENTION!
        gen = t2generator(name =cl.name+bly.name, block = blockname, type ='WATE', gx = m, ex = 1900.0E3)
        dat.add_generator(gen)
    #========================================

dat.parameter['tstop']=(3600.*24.*365.)*1500000. #<<<<<<TIMAX<<<<<<
dat.write('flow.inp')

mv: SAVE: No such file or directory


In [ ]:
#>>>>>>>>>>>>>>>>>>>ATENCIÓN>>>>>>>>>>>>>>>>>>>>>>>>>>>
#  con esta instrucción se ejecuta toughreact en mac
#  revisa cómo puedes ejecutar el software en tu sistema
#>>>>>>>>>>>>>>>>>>>ATENCIÓN>>>>>>>>>>>>>>>>>>>>>>>>>>>
os.system('export OMP_NUM_THREADS=1;./treactv332omp_eos3_macos')

 
      -------- TOUGHREACT Version 3.32-OMP ------- 
           Lawrence Berkeley National Laboratory   
                     February 8, 2017              
 
     ***** OpenMP Parallel Code Settings **** 
    *** maximum number of threads allotted  =   1
    *** maximum number of threads available =   8
 
    --> reading multiphase flow input data
 
    --> performing simulation


In [34]:
#Análisis de resultados - crea un objeto t2listing a partir del archivo flow.out
lst = t2listing('flow.out')

#posiciónate en la última tabla de la serie de tiempo
lst.last()
#imprime qué tiempo es - ¿correspode con el tiempo de simulación esperado?
print(lst.time/(3600*24*365))

#imprime la tabla de resultados del tiempo en el que 'last'
df=lst.element.DataFrame
#print(df.to_string())
#----------

1499873.160832065


In [28]:
#lst.convergence['P']
lst.convergence
#lst.reductions
#lst.times

{'P': (-51000.0, ' ab13'),
 'T': (0.02200000000001978, ' ac13'),
 'SG': (0.0, '  a 0'),
 'SL': (0.0, '  a 0'),
 'XAIRG': (0.0, '  a 0'),
 'XAIRL': (8.589000000000004e-08, '  e 2'),
 'PER.MOD.': (0.0, '  a 0'),
 'PCAP': (0.0, '  a 0'),
 'DG': (0.0, '  a 0'),
 'DL': (-0.04200000000003001, ' ac13')}

In [35]:
#Salida a Paraview cuando 'print_level=1'
#lst.write_vtk(geo,cwd+'/ParaViewOut/flow_outParaView.pvd', grid=t2geo, indices=None, flows=False, wells=False, start_time=0, time_unit='s', flux_matrix=None, blockmap = {}, surface_snap=0.1)

#Salida a Paraview cuando 'print_level=2'
lst.write_vtk(geo,cwd+'/ParaViewOut/flow_outParaView.pvd', grid=t2geo, indices=None, flows=True, wells=False, start_time=0, time_unit='s', flux_matrix=None, blockmap = {}, surface_snap=0.1)